In [ ]:
import numpy as np
import pandas as pd
import uproot
import awkward as ak
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def load(path):
    r = uproot.open(path)
    a = r["material-tracks"].arrays([
        "event_id",
        "v_eta",
        "v_phi",
        "mat_x",
        "mat_y",
        "mat_z",
        "mat_r",
        "mat_step_length",
        "mat_X0",
    ], library="ak")
    a = a[:100000]
    df = ak.to_dataframe(a, how="inner")
    return df

fatras = load("/Users/andreas/cern/thesis/data/odd-performance/scan/fatras_big/material_tracks.root")
geant4 = load("/Users/andreas/cern/thesis/data/odd-performance/scan/geant4_big/material_tracks.root")

In [ ]:
def accumulate(df):
    df["path"] = np.linalg.norm(df[["mat_x", "mat_y", "mat_z"]], axis=1)
    df["XoverX0"] = df["mat_step_length"] / df["mat_X0"]
    df["XoverX0_acc"] = df.groupby("event_id")["XoverX0"].transform(pd.Series.cumsum)
    df["XoverX0_acc_pre"] = df["XoverX0_acc"] - df["XoverX0"]

accumulate(fatras)
accumulate(geant4)

In [ ]:
event = 0

mask_fatras = (fatras["event_id"] == event) & (fatras["mat_r"] < 1150) & (fatras["mat_z"].abs() < 3000)
mask_geant4 = (geant4["event_id"] == event) & (geant4["mat_r"] < 1150) & (geant4["mat_z"].abs() < 3000)

eta = fatras[mask_fatras].iloc[0]["v_eta"]
print(f"eta fatras = {eta}")
eta = geant4[mask_geant4].iloc[0]["v_eta"]
print(f"eta geant4 = {eta}")

fig, ax = plt.subplots(1, 1)

ax.set_xlabel("path [mm]")
ax.set_ylabel("X/X0 []")

ax.scatter(fatras[mask_fatras]["path"], fatras[mask_fatras]["XoverX0_acc"], label="fatras")
ax.scatter(geant4[mask_geant4]["path"], geant4[mask_geant4]["XoverX0_acc"], label="geant4")

ax.legend();

In [ ]:
eta_range = (-0.1, 0.1)
phi_range = (-np.pi, np.pi)

def mask(df):
    return (
        (df["v_eta"] >= eta_range[0]) &
        (df["v_eta"] < eta_range[1]) &
        (df["v_phi"] >= phi_range[0]) &
        (df["v_phi"] < phi_range[1]) &
        (df["mat_r"] < 1150) &
        (df["mat_z"].abs() < 3000)
    )

mask_fatras = mask(fatras)
mask_geant4 = mask(geant4)

print(f"{eta_range[0]:.2f} <= eta < {eta_range[1]:.2f}")
print(f"{phi_range[0]:.2f} <= phi < {phi_range[1]:.2f}")

fig, ax = plt.subplots(1, 1)

ax.set_xlabel("path [mm]")
ax.set_ylabel("X/X0 []")

ax.scatter(fatras[mask_fatras]["path"], fatras[mask_fatras]["XoverX0_acc"], alpha=0.2, zorder=1, label="fatras")
ax.scatter(geant4[mask_geant4]["path"], geant4[mask_geant4]["XoverX0_acc"], alpha=0.2, zorder=0, label="geant4")

ax.legend();

In [ ]:
def aggregate_region(df, r_range, z_range):
    mask = (
        (df["mat_r"] >= r_range[0]) &
        (df["mat_r"] < r_range[1]) &
        (df["mat_z"] >= z_range[0]) &
        (df["mat_z"] < z_range[1])
    )

    return df[mask].groupby("event_id").agg({
        "v_eta": "last",
        "v_phi": "last",
        "path": "last",
        "XoverX0_acc": "last",
        "XoverX0_acc_pre": "first",
    })

def plot_mean(ax, df, eta_edges, phi_edges, vmin=0, vmax=0.05):
    mean, _, _, _ = scipy.stats.binned_statistic_2d(
        df["v_eta"],
        df["v_phi"],
        df["XoverX0_acc"] - df["XoverX0_acc_pre"],
        statistic="mean",
        bins=(eta_edges, phi_edges),
    )

    eta_mid = 0.5 * (eta_edges[:-1] + eta_edges[1:])
    phi_mid = 0.5 * (phi_edges[:-1] + phi_edges[1:])

    sns.heatmap(
        mean.T,
        #annot=True,
        cmap="plasma",
        vmin=vmin,
        vmax=vmax,
        ax=ax,
    )
    ax.set_xlabel("eta")
    ax.set_ylabel("phi")
    ax.invert_yaxis()

    return mean

def plot_mean_cmp(axs, df1, df2, eta_edges, phi_edges, vmin=0, vmax=0.05):
    df1_mean = plot_mean(axs[0], df1, eta_edges, phi_edges, vmin=vmin, vmax=vmax)
    df2_mean = plot_mean(axs[1], df2, eta_edges, phi_edges, vmin=vmin, vmax=vmax)

    eta_mid = 0.5 * (eta_edges[:-1] + eta_edges[1:])
    phi_mid = 0.5 * (phi_edges[:-1] + phi_edges[1:])

    sns.heatmap(
        (df2_mean / df1_mean).T,
        #annot=True,
        cmap="viridis",
        ax=axs[2],
    )
    axs[2].set_xlabel("eta")
    axs[2].set_ylabel("phi")
    axs[2].invert_yaxis()

In [ ]:
pixel_barrel_z_range = (-500, 500)
strip_barrel_z_range = (-1140, 1140)
regions = {
    "beampipe": {"r_range": (0, 27), "z_range": (-3000, 3000)},
    "pixel_barrel_layer0": {"r_range": (30, 40), "z_range": pixel_barrel_z_range},
    "pixel_barrel_layer1": {"r_range": (65, 80), "z_range": pixel_barrel_z_range},
    "pixel_barrel_layer2": {"r_range": (110, 125), "z_range": pixel_barrel_z_range},
    "pixel_barrel_layer3": {"r_range": (165, 180), "z_range": pixel_barrel_z_range},
    "strip_barrel_layer0": {"r_range": (220, 280), "z_range": strip_barrel_z_range},
    "strip_barrel_layer1": {"r_range": (320, 380), "z_range": strip_barrel_z_range},
    "strip_barrel_layer2": {"r_range": (460, 520), "z_range": strip_barrel_z_range},
    "strip_barrel_layer3": {"r_range": (620, 680), "z_range": strip_barrel_z_range},
}

In [ ]:
eta_range = (-1, 1)
phi_range = (-np.pi, np.pi)

def mask(df):
    return (
        (df["v_eta"] >= eta_range[0]) &
        (df["v_eta"] < eta_range[1]) &
        (df["v_phi"] >= phi_range[0]) &
        (df["v_phi"] < phi_range[1])
    )

mask_fatras = mask(fatras)
mask_geant4 = mask(geant4)

eta_edges = np.linspace(*eta_range, 8)
phi_edges = np.linspace(*phi_range, 100)

fig, axs = plt.subplots(4, 3, figsize=(12, 12))
i = 0
for name, config in regions.items():
    r_range, z_range = config["r_range"], config["z_range"]
    if not name.startswith("pixel_barrel_"):
        continue

    fatras_agg = aggregate_region(fatras[mask_fatras], r_range, z_range)
    geant4_agg = aggregate_region(geant4[mask_geant4], r_range, z_range)

    plot_mean_cmp(axs[i], fatras_agg, geant4_agg, eta_edges, phi_edges)

    i += 1

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(12, 12))
i = 0
for name, config in regions.items():
    r_range, z_range = config["r_range"], config["z_range"]
    if not name.startswith("strip_barrel_"):
        continue

    fatras_agg = aggregate_region(fatras[mask_fatras], r_range, z_range)
    geant4_agg = aggregate_region(geant4[mask_geant4], r_range, z_range)

    plot_mean_cmp(axs[i], fatras_agg, geant4_agg, eta_edges, phi_edges)

    i += 1

In [ ]:
# upper bound r
limits = {
    "beampipe": {"r_range": (0, 27), "z_range": (-3000, 3000), "v_range": (0, 0.02)},
    "pixel_barrel_layer0": {"r_range": (0, 36.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.02)},
    "pixel_barrel_layer1": {"r_range": (0, 73.0), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "pixel_barrel_layer2": {"r_range": (0, 119), "z_range": pixel_barrel_z_range, "v_range": (0, 0.06)},
    "pixel_barrel_layer3": {"r_range": (0, 175), "z_range": pixel_barrel_z_range, "v_range": (0, 0.08)},
    "strip_barrel_layer0": {"r_range": (0, 266), "z_range": strip_barrel_z_range, "v_range": (0, 0.02)},
    "strip_barrel_layer1": {"r_range": (0, 366), "z_range": strip_barrel_z_range, "v_range": (0, 0.02)},
    "strip_barrel_layer2": {"r_range": (0, 505), "z_range": strip_barrel_z_range, "v_range": (0, 0.02)},
    "strip_barrel_layer3": {"r_range": (0, 666), "z_range": strip_barrel_z_range, "v_range": (0, 0.02)},
}

# lower bound r
limits = {
    "beampipe": {"r_range": (0, 27), "z_range": (-3000, 3000), "v_range": (0, 0.02)},
    "pixel_barrel_layer0": {"r_range": (0, 31.0), "z_range": pixel_barrel_z_range, "v_range": (0, 0.02)},
    "pixel_barrel_layer1": {"r_range": (0, 67.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "pixel_barrel_layer2": {"r_range": (0, 113.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.06)},
    "pixel_barrel_layer3": {"r_range": (0, 169.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.08)},
    "strip_barrel_layer0": {"r_range": (0, 251), "z_range": strip_barrel_z_range, "v_range": (0, 0.10)},
    "strip_barrel_layer1": {"r_range": (0, 350), "z_range": strip_barrel_z_range, "v_range": (0, 0.12)},
    "strip_barrel_layer2": {"r_range": (0, 491), "z_range": strip_barrel_z_range, "v_range": (0, 0.14)},
    "strip_barrel_layer3": {"r_range": (0, 651), "z_range": strip_barrel_z_range, "v_range": (0, 0.16)},
}

# layer to layer
limits = {
    "pixel_barrel_layer0": {"r_range": (0, 31.0), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "pixel_barrel_layer1": {"r_range": (36.5, 67.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "pixel_barrel_layer2": {"r_range": (73.0, 113.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "pixel_barrel_layer3": {"r_range": (119, 169.5), "z_range": pixel_barrel_z_range, "v_range": (0, 0.04)},
    "strip_barrel_layer0": {"r_range": (175, 251), "z_range": strip_barrel_z_range, "v_range": (0, 0.08)},
    "strip_barrel_layer1": {"r_range": (266, 350), "z_range": strip_barrel_z_range, "v_range": (0, 0.08)},
    "strip_barrel_layer2": {"r_range": (366, 491), "z_range": strip_barrel_z_range, "v_range": (0, 0.08)},
    "strip_barrel_layer3": {"r_range": (505, 651), "z_range": strip_barrel_z_range, "v_range": (0, 0.08)},
}

In [ ]:
eta_range = (-1, 1)
phi_range = (-np.pi, np.pi)

def mask(df):
    return (
        (df["v_eta"] >= eta_range[0]) &
        (df["v_eta"] < eta_range[1]) &
        (df["v_phi"] >= phi_range[0]) &
        (df["v_phi"] < phi_range[1])
    )

mask_fatras = mask(fatras)
mask_geant4 = mask(geant4)

eta_edges = np.linspace(*eta_range, 8)
phi_edges = np.linspace(*phi_range, 100)

fig, axs = plt.subplots(4, 3, figsize=(12, 12))
i = 0
for name, config in limits.items():
    r_range, z_range, v_range = config["r_range"], config["z_range"], config["v_range"]
    if not name.startswith("pixel_barrel_"):
        continue

    fatras_agg = aggregate_region(fatras[mask_fatras], r_range, z_range)
    geant4_agg = aggregate_region(geant4[mask_geant4], r_range, z_range)

    plot_mean_cmp(axs[i], fatras_agg, geant4_agg, eta_edges, phi_edges, vmin=v_range[0], vmax=v_range[1])

    i += 1

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(12, 12))
i = 0
for name, config in limits.items():
    r_range, z_range, v_range = config["r_range"], config["z_range"], config["v_range"]
    if not name.startswith("strip_barrel_"):
        continue

    fatras_agg = aggregate_region(fatras[mask_fatras], r_range, z_range)
    geant4_agg = aggregate_region(geant4[mask_geant4], r_range, z_range)

    plot_mean_cmp(axs[i], fatras_agg, geant4_agg, eta_edges, phi_edges, vmin=v_range[0], vmax=v_range[1])

    i += 1